In [1]:
import webdataset as wds
import os
from tqdm import tqdm
import glob
import numpy as np

In [2]:
# data_name = 'CARRADA'
data_name = 'RADDet'
fold_idx = '01'

if data_name == 'CARRADA':
    data_dir = '/mnt/SrvDataDisk/Datasets_Radar/CARRADA'
    sequences_all = glob.glob(os.path.join(data_dir, "Carrada_RAD", "*/*/*.npy"))
elif data_name == 'RADDet':
    data_dir = '/mnt/Disk/zx/data/RADDet_author'
    sequences_train = glob.glob(os.path.join(data_dir, "train", "RAD/*/*.npy"))
    sequences_test = glob.glob(os.path.join(data_dir, "test", "RAD/*/*.npy"))
    sequences_all = sequences_train + sequences_test
sequences_all.sort()

index_start = 0
index_end = 10
# 0-1000, 1000-2001, 2001-3001, 3001-4001, 4001-5001, 5001-6001, 6001-7001, 7001-8001, 8001-9001, 9001-:
sequence = sequences_all[index_start:index_end]

sink = wds.TarWriter(f"{data_name}_{fold_idx}.tar") # 使用TarWriter，准备将数据写入.tar文件

index = index_start
for filename in tqdm(sequence):
    sample = np.load(filename)
    data = {
        "__key__": f"sample_{index:06d}",  # 当前样本的index
        "input.npy": sample,  # 雷达数据
    }
    sink.write(data) # 将数据写入.tar文件
    index += 1
sink.close() # 关闭.tar文件

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


KeyboardInterrupt: 

In [3]:
from torch.utils.data import DataLoader
from io import BytesIO

def decode_npy(sample):
    # sample 是一个包含相关信息的字典
    byte_data = sample['input.npy']  # 获取字节数据
    # 使用 BytesIO 将字节数据转换为文件对象
    data = np.load(BytesIO(byte_data))  # 加载 .npy 文件

    index = sample['__key__']  # 获取索引信息
    return index, data

dataset = (
    # wds.WebDataset("CARRADA_{01..02}.tar", shardshuffle=True)
    wds.WebDataset("CARRADA_01.tar", shardshuffle=True)
    .shuffle(100)
    .map(decode_npy)
    .with_epoch(10)
    .with_length(100)
    # .decode()
    # .to_tuple("input.npy")
)
batch = next(iter(dataset))

dataloader = DataLoader(
    dataset, 
    batch_size=1, 
    # shuffle=True,     # 这里不能用shuffle？
    num_workers=2  
)
for idx, batch in dataloader:
    print(idx)
    print(batch.shape)




('sample_000053',)
torch.Size([1, 256, 256, 64])


ValueError: Caught ValueError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 33, in fetch
    data.append(next(self.dataset_iter))
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/webdataset/pipeline.py", line 71, in iterator
    for sample in self.iterator1():
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/webdataset/filters.py", line 305, in _map
    for sample in data:
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/webdataset/filters.py", line 217, in _shuffle
    for sample in data:
  File "/home/ljm/anaconda3/envs/rpt/lib/python3.10/site-packages/webdataset/compat.py", line 105, in check_empty
    rais

In [1]:
index_list = []
gap = 500
len_all = 12666
for start in range(0, len_all, gap):
    end = min(start + gap, len_all)  # 确保不会超出范围
    index_list.append([start, end])

print(index_list)

[[0, 500], [500, 1000], [1000, 1500], [1500, 2000], [2000, 2500], [2500, 3000], [3000, 3500], [3500, 4000], [4000, 4500], [4500, 5000], [5000, 5500], [5500, 6000], [6000, 6500], [6500, 7000], [7000, 7500], [7500, 8000], [8000, 8500], [8500, 9000], [9000, 9500], [9500, 10000], [10000, 10500], [10500, 11000], [11000, 11500], [11500, 12000], [12000, 12500], [12500, 12666]]
